In [119]:
import pandas as pd
import ray
import requests
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin, urlparse
from tqdm import tqdm

In [111]:
PATH_URLS = "datahuck_sample.csv"

urls_df = pd.read_csv(PATH_URLS, index_col="seed_url")

In [131]:
class Crawler:
    def __init__(self, output_path="output.csv"):
        self.output_path = output_path
        self.errors_path = f"errors_{output_path}"

        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.3'}

        self.num_crawled = 0 # crawled is defined as succesfully extracted relevant information
        self.num_errors = 0
    
    def __str__(self):
        return f'Crawler has so far processed [{self.num_crawled}] urls and failed on [{self.num_errors}] urls'
    
    def process_url(self, url):
        # TODO check Here
        url = url.replace("https", "http")
        try:
            response = requests.get(url, headers=self.headers)

            response.raise_for_status()  # Check if the request was successful

            soup = BeautifulSoup(response.content, 'html.parser')
            # Find all <a> tags with an href attribute
            links = soup.find_all('a', href=True)
            
            # Extract the href attribute from each <a> tag and store it in a list
            hrefs = [link['href'] for link in links]
            hrefs = [urljoin(url, link['href']) for link in links]
            hrefs = [link.replace(urlparse(link).scheme + ':', 'http:') for link in hrefs]

            # To see all internal links
            # print([l for l in hrefs if url in l])

            hrefs = [urlparse(link).path.strip("/") for link in hrefs if url in link]
            hrefs = list(set([l for l in hrefs if len(l) > 0]))

            self.num_crawled += 1

            return {"url": url, "results": hrefs, "error?": False}

        except requests.RequestException as e:
            self.num_errors += 1
            return {"url": url, "results": e, "error?": True}

    def batch_process(self, urls, batch_size=10, alias="ALL"):
        """
        Process a urls in batches

        urls: pd.DataFrame of urls
        batch_size: size of batch
        alias: When parallel processing give each instance a unique identitifier so intermediate results don't clash
        """
        num_batches = len(df) // batch_size + (len(df) % batch_size > 0)
        processed_data = []
        print(f'Processing {alias} with batch size: {batch_size} and number of batches: {num_batches}')

        for i in tqdm(range(num_batches)):
            batch = urls.iloc[i * batch_size:(i + 1) * batch_size]
            for seed_url in batch.index:
                processed_data.append(self.process_url(seed_url))
                
            # Save intermediate results
            intermediate_df = pd.DataFrame(processed_data)
            intermediate_df.to_csv(f'{alias}_intermediate_results.csv', index=False)

        # Save final results
        final_df = pd.DataFrame(processed_data)
        final_df.to_csv(f'{alias}_{self.output_path}', index=False)
        print(self)
        return final_df

In [132]:
c = Crawler()
c.batch_process(urls_df.head(100))

Processing ALL with batch size: 10 and number of batches: 100


100%|██████████| 100/100 [04:16<00:00,  2.57s/it]

Crawler has so far processed [95] urls and failed on [5] urls


,url,results,error?
0,http://nataliehuggins.com,"[media, yoga, yoga-classes-workshops, store, a...",False
1,http://fauxynaturalhaircare.com,"[collections/all, products/shampoo, products/d...",False
2,http://winkeyless.kr,"[product/bold-case-mini, forums/topic/possible...",False
3,http://anna-goodman.com,"[collections/chakra-set, cdn/shop/products/ANN...",False
4,http://darrenbooth.com,"[info, work, newsletter, anobjectofbeauty, let...",False
...,...,...,...
95,http://www.chimytina.com,"[videos, donate, contact, about, press, cart]",False
96,http://moisturelove.com,"[pages/reviews, products/stimulating-growth-oi...",False
97,http://www.rauchstone.com,[],False
98,http://melhodge.com,"[home/project-six-pnpma, home/project-one-pxrz...",False


In [136]:
import concurrent.futures

segments = p1, p2  = urls_df.iloc[0:100], urls_df.iloc[100: 200]

# Create instances of Crawler for each DataFrame
crawler_instances = [Crawler() for _ in segments]

# Function to wrap batch_process method
def process_with_crawler(crawler, df, alias):
    return crawler.batch_process(df, alias=alias)

with concurrent.futures.ThreadPoolExecutor(max_workers=len(segments)) as executor:
    futures = [executor.submit(process_with_crawler, crawler, df, f"DF{i+1}") for i, (crawler, df) in enumerate(zip(crawler_instances, segments))]
    
    # Wait for all futures to complete (optional)
    for future in concurrent.futures.as_completed(futures):
        pass  # We are ignoring the results

Processing DF1 with batch size: 10 and number of batches: 100
Processing DF2 with batch size: 10 and number of batches: 100


  4%|▍         | 4/100 [02:05<49:59, 31.24s/it]

In [40]:

# Initialize Ray
ray.init()

# Define a function to extract links from a single URL
@ray.remote
def extract_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [a.get('href') for a in soup.find_all('a', href=True)]
        return links
    except requests.RequestException as e:
        print(f"Request failed for {url}: {e}")
        return []

# Process URLs in batches
batch_size = 10
all_results = []

for i in range(0, len(df), batch_size):
    batch_urls = df.index[i:i + batch_size]
    results = ray.get([extract_links.remote(url) for url in batch_urls])
    all_results.extend(results)

# Combine results and create a new DataFrame
df['links'] = all_results

# Print the DataFrame with extracted links
print(df)

# Shut down Ray
ray.shutdown()


2024-07-25 15:30:45,046	INFO worker.py:1788 -- Started a local Ray instance.


(extract_links pid=25065) Request failed for http://tedxrexburg.com: 406 Client Error: Not Acceptable for url: http://tedxrexburg.com/
(extract_links pid=25070) Request failed for http://goslg.com: 406 Client Error: Not Acceptable for url: http://goslg.com/


(raylet) [2024-07-25 15:30:54,994 E 25061 28995314] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-25_15-30-43_457913_24735 is over 95% full, available space: 18624638976; capacity: 494384795648. Object creation will fail if spilling is required.


(extract_links pid=25074) Request failed for http://poopieandpooper.com: 404 Client Error: Not Found for url: http://poopieandpooper.com/
(extract_links pid=25067) Request failed for http://wiserpiercing.com: 404 Client Error: Not Found for url: http://wiserpiercing.com/


(raylet) [2024-07-25 15:31:05,093 E 25061 28995314] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-25_15-30-43_457913_24735 is over 95% full, available space: 18623680512; capacity: 494384795648. Object creation will fail if spilling is required.


...

(extract_links pid=25073) Request failed for http://bestbuytrading.in: HTTPConnectionPool(host='bestbuytrading.in', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x107fe3b50>: Failed to establish a new connection: [Errno 64] Host is down'))


(raylet) [2024-07-25 15:45:32,201 E 25061 28995314] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-25_15-30-43_457913_24735 is over 95% full, available space: 18625855488; capacity: 494384795648. Object creation will fail if spilling is required.


(extract_links pid=25067) Request failed for http://www.dawnbrolin.com: 403 Client Error: Forbidden for url: http://www.dawnbrolin.com/
(extract_links pid=25069) Request failed for http://www.kool4u.com.au: HTTPConnectionPool(host='www.kool4u.com.au', port=80): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x110b3a310>: Failed to resolve 'www.kool4u.com.au' ([Errno 8] nodename nor servname provided, or not known)"))


(raylet) [2024-07-25 15:45:42,293 E 25061 28995314] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2024-07-25_15-30-43_457913_24735 is over 95% full, available space: 18625880064; capacity: 494384795648. Object creation will fail if spilling is required.
/var/folders/9f/_6775r3x22z0p4kb6_fn2xmr0000gn/T/ipykernel_24735/1809471278.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['links'] = all_results


                                                                             links
seed_url                                                                          
http://nataliehuggins.com        [#page, /, /about-1, /epk, /lessons, /yoga, /e...
http://fauxynaturalhaircare.com  [#MainContent, /cart, /collections/all, /, /, ...
http://winkeyless.kr             [#site-navigation, #content, https://winkeyles...
http://anna-goodman.com          [#MainContent, /, /collections, /collections/c...
http://darrenbooth.com           [/cart, #page, /, /work, /illustration, /lette...
...                                                                            ...
http://invernoscent.com          [#site-main, /, #, #, #, /account/login, /cart...
http://siobeauty.com             [#main-content, /collections/smoothing-silicon...
http://simonclaridge.com         [/, /cart, /search, /, /new, /shop, /about, /s...
http://carpediemmarkers.com      [/, /register?returnUrl=%2F, /login?returnUrl=...
http

In [44]:
def extract_links(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check if the request was successful
        soup = BeautifulSoup(response.content, 'html.parser')
        links = [a.get('href') for a in soup.find_all('a', href=True)]
        return links
    except requests.RequestException as e:
        print(f"Request failed for {url}: {e}")
        return []
# 402 error
# https://paprkut.com/

# 403 error 
# http://www.dawnbrolin.com

# 404 error
# http://lovelikewhiskey.com

# 406 error
# http://ionacraftshop.com

# 500 error (unsolvable)
# http://howmanysyrians.com

# 444 error
# http://narahita.com

# connection aborted
# http://www.plusgami.com

# Max retries (unsolvable)
# http://lovelikewhiskey.com

